# Libraries

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader.data as web
import tensorflow as tf 
from tensorflow import keras 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Concatenate
from keras import optimizers
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Model with All Drivers 

In [4]:
def data(ticker,start,end):
    df = web.DataReader(ticker.upper(),'yahoo',start,end)[['Adj Close','High','Low']]
    df['sm3'] = df['Adj Close'].rolling(window=3).mean() #3 day moving average
    df['sm9'] = df['Adj Close'].rolling(window=9).mean() #9 day moving average
    drivers = ['MSFT','AAPL','GOOG','GOOGL','AMZN','FB','^VIX'] #make a large portion of the etf's holdings
    portfolio = pd.DataFrame() #create an emy dataframe to place data 
    failed = [] #empty list for failed queries
    for stock in drivers: #for loop for all the features I want to try
        try: #try clause just in case it failed
            portfolio[stock] = web.DataReader(stock,'yahoo',start,end)['Adj Close'] #adj close of each feature
            #moving averages to normalize the features
            portfolio[f'{stock} sma3'] = portfolio[stock].rolling(window=3).mean() #moving averages as features
            portfolio[f'{stock} sma9'] = portfolio[stock].rolling(window=9).mean()
            portfolio = portfolio.drop([stock],axis=1) #drop the adj close price
        except:
            failed.append(stock) #add failed query ticker to list
            print(f'{failed} was not properly calculated for. Are you sure this ticker is on an exchange?')
    data = pd.concat([df,portfolio],axis = 1).dropna() #combine both dataframes
    data['Target'] = data['Adj Close'].shift(-1) #shift adj close back 1: we are forecasting one day into the future
    data = data.drop(['Adj Close','High','Low'],axis =1) #drop these bc nn works better 
    return data.dropna() #no null values



def predict(df,pred_df):
    X = df[df.columns] #feature data
    del X['Target'] #don't want target variable in training data
    Y = df[['Target']] #label data
    x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=50,test_size=0.2) #training and testing
    x_val,x_test,y_val,y_test = train_test_split(x_test,y_test,random_state=50,test_size=0.5) #test and validation
    model = Sequential() #basic sequential model 
    model.add(Dense(100, input_dim=x_train.shape[1], #need the input shape of the data in tensorflow 2x
                        activation=tf.nn.leaky_relu, # was better than relu
                        kernel_initializer='he_normal'))
    model.add(Dense(75, input_dim=100, #100 'neurons' in the input layer
                        activation=tf.nn.leaky_relu,
                        kernel_initializer='he_normal'))
    model.add(Dense(50, input_dim=75, #75 'neurons' in the first hidden layer
                    activation=tf.nn.leaky_relu,
                    kernel_initializer='he_normal'))
    model.add(Dense(25, input_dim=50, #50 'neurons' in the second hidden layer
                    activation=tf.nn.leaky_relu,
                    kernel_initializer='he_normal'))
    model.add(Dense(1, activation=tf.nn.leaky_relu, #only one answer so you need one 'neuron'
                        kernel_initializer='he_normal'))
    model.compile(loss='mean_squared_error', #mse loss function
                      optimizer='adam', #adam optimizer
                      metrics=['mape']) #mean absolute percentage error metric to determine the performance of the model
    scaler = MinMaxScaler() #normalize the data since it is pretty different in terms of share price
    x_train_scaled = scaler.fit_transform(x_train) #apply the normalizer to the training features
    history = model.fit(x_train, y_train,  #fit the training data to the model
                        validation_data=(x_val, y_val), #validation data to better see how the model is doing
                        batch_size=32,
                        epochs=10,
                        verbose=0)
    #pred_df = pred[features.columns]
    pred_features = pred_df.iloc[-1] #these will be in the pred_data function 
    prediction = model.predict(np.array([pred_features])) #need it in numpy array 
    print(f'The predicted stock price for {ticker.upper()} tomorrow is ${float(prediction[0])}.')
    if float(prediction[0])>float(df['Target'].iloc[-1:].values):
        print('Buy: ', float(prediction[0]), '>',float(df['Target'].iloc[-1:].values),'\nPCT DIFF: ',(float(prediction[0]) - float(df['Target'].iloc[-1:].values))/float(df['Target'].iloc[-1:].values)*100,'%',)
    else:
        print('Sell: ', float(prediction[0]), '<',float(df['Target'].iloc[-1:].values),'\nPCT DIFF: ',(float(prediction[0]) - float(df['Target'].iloc[-1:].values))/float(df['Target'].iloc[-1:].values)*100,'%')



def pred_data(ticker,start,end):
    df = web.DataReader(ticker.upper(),'yahoo',start,end)[['Adj Close']] #query data of target security
    df['sm3'] = df['Adj Close'].rolling(window=3).mean() #moving averages
    df['sm9'] = df['Adj Close'].rolling(window=9).mean()
    drivers = ['MSFT','AAPL','GOOG','GOOGL','AMZN','FB','^VIX'] #drivers 
    failed = []
    portfolio = pd.DataFrame() #dataframe to add data to during the for loop
    for stock in drivers: #for loop
        try: #try clause
            portfolio[stock] = web.DataReader(stock,'yahoo',start,end)['Adj Close']
            portfolio[f'{stock} sma3'] = portfolio[stock].rolling(window=3).mean()
            portfolio[f'{stock} sma9'] = portfolio[stock].rolling(window=9).mean()
            portfolio = portfolio.drop([stock],axis=1) #drop adj close
        except:
            failed.append(stock) #add to failed list to see which ticker failed
            print(f'{failed} was not properly calculated for. Are you sure this ticker is on an exchange?')
    data = pd.concat([df,portfolio],axis = 1).dropna() #combine data lists
    data = data.drop(['Adj Close'],axis = 1)
    return data.dropna()

        
        
if __name__ == '__main__':
    ticker = 'spy' #target security
    start = dt.datetime.now() - dt.timedelta(days=365*5) #5 year time frame
    end = dt.datetime.now() #today
    predict(data(ticker,start,end),pred_data(ticker,start,end)) #calling the function we want to predict the target security

The predicted stock price for SPY tomorrow is $372.0769348144531.
Buy:  372.0769348144531 > 357.4599914550781 
PCT DIFF:  4.0891131060220784 %


# Model with Just Moving Averages

In [6]:
def data(ticker,start,end):
    df = web.DataReader(ticker.upper(),'yahoo',start,end)[['Adj Close','High','Low']]
    df['sm3'] = df['Adj Close'].rolling(window=3).mean() #3 day moving average
    df['sm9'] = df['Adj Close'].rolling(window=9).mean() #9 day moving average
    df['Target'] = df['Adj Close'].shift(-1) #shift adj close back 1: we are forecasting one day into the future
    df = df.drop(['Adj Close','High','Low'],axis =1) #drop these bc nn works better 
    return df.dropna() #no null values



def predict(df,pred_df):
    X = df[df.columns] #feature data
    del X['Target'] #don't want target variable in training data
    Y = df[['Target']] #label data
    x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=50,test_size=0.2) #training and testing
    x_val,x_test,y_val,y_test = train_test_split(x_test,y_test,random_state=50,test_size=0.5) #test and validation
    model = Sequential() #basic sequential model 
    model.add(Dense(100, input_dim=x_train.shape[1], #need the input shape of the data in tensorflow 2x
                        activation=tf.nn.leaky_relu, # was better than relu
                        kernel_initializer='he_normal'))
    model.add(Dense(75, input_dim=100, #100 'neurons' in the input layer
                        activation=tf.nn.leaky_relu,
                        kernel_initializer='he_normal'))
    model.add(Dense(50, input_dim=75, #75 'neurons' in the first hidden layer
                    activation=tf.nn.leaky_relu,
                    kernel_initializer='he_normal'))
    model.add(Dense(25, input_dim=50, #50 'neurons' in the second hidden layer
                    activation=tf.nn.leaky_relu,
                    kernel_initializer='he_normal'))
    model.add(Dense(1, activation=tf.nn.leaky_relu, #only one answer so you need one 'neuron'
                        kernel_initializer='he_normal'))
    model.compile(loss='mean_squared_error', #mse loss function
                      optimizer='adam', #adam optimizer
                      metrics=['mape']) #mean absolute percentage error metric to determine the performance of the model
    scaler = MinMaxScaler() #normalize the data since it is pretty different in terms of share price
    x_train_scaled = scaler.fit_transform(x_train) #apply the normalizer to the training features
    history = model.fit(x_train, y_train,  #fit the training data to the model
                        validation_data=(x_val, y_val), #validation data to better see how the model is doing
                        batch_size=32,
                        epochs=20,
                        verbose=1)
    #pred_df = pred[features.columns]
    pred_features = pred_df.iloc[-1] #these will be in the pred_data function 
    prediction = model.predict(np.array([pred_features])) #need it in numpy array 
    print(f'The predicted stock price for {ticker.upper()} tomorrow is ${float(prediction[0])}.')
    if float(prediction[0])>float(df['Target'].iloc[-1:].values):
        print('Buy: ', float(prediction[0]), '>',float(df['Target'].iloc[-1:].values),'\nPCT DIFF: ',(float(prediction[0]) - float(df['Target'].iloc[-1:].values))/float(df['Target'].iloc[-1:].values)*100,'%',)
    else:
        print('Sell: ', float(prediction[0]), '<',float(df['Target'].iloc[-1:].values),'\nPCT DIFF: ',(float(prediction[0]) - float(df['Target'].iloc[-1:].values))/float(df['Target'].iloc[-1:].values)*100,'%')



def pred_data(ticker,start,end):
    df = web.DataReader(ticker.upper(),'yahoo',start,end)[['Adj Close']] #query data of target security
    df['sm3'] = df['Adj Close'].rolling(window=3).mean() #moving averages
    df['sm9'] = df['Adj Close'].rolling(window=9).mean()
    df = df.drop(['Adj Close'],axis = 1)
    return df.dropna()

        
        
if __name__ == '__main__':
    ticker = 'spy' #target security
    start = dt.datetime.now() - dt.timedelta(days=365*5) #5 year time frame
    end = dt.datetime.now() #today
    predict(data(ticker,start,end),pred_data(ticker,start,end)) #calling the function we want to predict the target security

Epoch 1/20
32/32 [==============================] - 1s 14ms/step - loss: 34578.0473 - mape: 57.8134 - val_loss: 920.7321 - val_mape: 11.5423
Epoch 2/20
32/32 [==============================] - 0s 5ms/step - loss: 445.5009 - mape: 7.2259 - val_loss: 64.7284 - val_mape: 2.7398
Epoch 3/20
32/32 [==============================] - 0s 5ms/step - loss: 44.7541 - mape: 1.9697 - val_loss: 25.5212 - val_mape: 1.2413
Epoch 4/20
32/32 [==============================] - 0s 5ms/step - loss: 33.2069 - mape: 1.4047 - val_loss: 25.2203 - val_mape: 1.2134
Epoch 5/20
32/32 [==============================] - 0s 5ms/step - loss: 25.7674 - mape: 1.2849 - val_loss: 35.9312 - val_mape: 1.5912
Epoch 6/20
32/32 [==============================] - 0s 7ms/step - loss: 29.0562 - mape: 1.4272 - val_loss: 25.9267 - val_mape: 1.2243
Epoch 7/20
32/32 [==============================] - 0s 5ms/step - loss: 21.1162 - mape: 1.2041 - val_loss: 42.7701 - val_mape: 2.1150
Epoch 8/20
32/32 [==============================] - 0s

# Model with Moving Averages and High / Low MA strategy

In [12]:
def data(ticker,start,end):
    df = web.DataReader(ticker.upper(),'yahoo',start,end)[['Adj Close','High','Low']]
    df['sm3'] = df['Adj Close'].rolling(window=3).mean() #3 day moving average
    df['sm9'] = df['Adj Close'].rolling(window=9).mean() #9 day moving average
    df['HighSM'] = df['High'].rolling(window=5).mean() #high moving average
    df['LowSM'] = df['Low'].rolling(window=5).mean() #low moving average
    df['Target'] = df['Adj Close'].shift(-1) #shift adj close back 1: we are forecasting one day into the future
    df = df.drop(['Adj Close','High','Low'],axis =1) #drop these bc nn works better 
    return df.dropna() #no null values



def predict(df,pred_df):
    X = df[df.columns] #feature data
    del X['Target'] #don't want target variable in training data
    Y = df[['Target']] #label data
    x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=50,test_size=0.2) #training and testing
    x_val,x_test,y_val,y_test = train_test_split(x_test,y_test,random_state=50,test_size=0.5) #test and validation
    model = Sequential() #basic sequential model 
    model.add(Dense(100, input_dim=x_train.shape[1], #need the input shape of the data in tensorflow 2x
                        activation=tf.nn.leaky_relu, # was better than relu
                        kernel_initializer='he_normal'))
    model.add(Dense(75, input_dim=100, #100 'neurons' in the input layer
                        activation=tf.nn.leaky_relu,
                        kernel_initializer='he_normal'))
    model.add(Dense(50, input_dim=75, #75 'neurons' in the first hidden layer
                    activation=tf.nn.leaky_relu,
                    kernel_initializer='he_normal'))
    model.add(Dense(25, input_dim=50, #50 'neurons' in the second hidden layer
                    activation=tf.nn.leaky_relu,
                    kernel_initializer='he_normal'))
    model.add(Dense(1, activation=tf.nn.leaky_relu, #only one answer so you need one 'neuron'
                        kernel_initializer='he_normal'))
    model.compile(loss='mean_squared_error', #mse loss function
                      optimizer='adam', #adam optimizer
                      metrics=['mape']) #mean absolute percentage error metric to determine the performance of the model
    scaler = MinMaxScaler() #normalize the data since it is pretty different in terms of share price
    x_train_scaled = scaler.fit_transform(x_train) #apply the normalizer to the training features
    history = model.fit(x_train, y_train,  #fit the training data to the model
                        validation_data=(x_val, y_val), #validation data to better see how the model is doing
                        batch_size=32,
                        epochs=20,
                        verbose=1)
    #pred_df = pred[features.columns]
    pred_features = pred_df.iloc[-1] #these will be in the pred_data function 
    prediction = model.predict(np.array([pred_features])) #need it in numpy array 
    print(f'The predicted stock price for {ticker.upper()} tomorrow is ${float(prediction[0])}.')
    if float(prediction[0])>float(df['Target'].iloc[-1:].values):
        print('Buy: ', float(prediction[0]), '>',float(df['Target'].iloc[-1:].values),'\nPCT DIFF: ',(float(prediction[0]) - float(df['Target'].iloc[-1:].values))/float(df['Target'].iloc[-1:].values)*100,'%',)
    else:
        print('Sell: ', float(prediction[0]), '<',float(df['Target'].iloc[-1:].values),'\nPCT DIFF: ',(float(prediction[0]) - float(df['Target'].iloc[-1:].values))/float(df['Target'].iloc[-1:].values)*100,'%')



def pred_data(ticker,start,end):
    df = web.DataReader(ticker.upper(),'yahoo',start,end)[['Adj Close','High','Low']] #query data of target security
    df['sm3'] = df['Adj Close'].rolling(window=3).mean() #moving averages (3 day)
    df['sm9'] = df['Adj Close'].rolling(window=9).mean() #moving averages (9 day)
    df['HighSM'] = df['High'].rolling(window=5).mean() #high moving average
    df['LowSM'] = df['Low'].rolling(window=5).mean() #low moving average
    df = df.drop(['Adj Close','High','Low'],axis = 1)
    return df.dropna()

        
        
if __name__ == '__main__':
    ticker = 'spy' #target security
    start = dt.datetime.now() - dt.timedelta(days=365*5) #5 year time frame
    end = dt.datetime.now() #today
    predict(data(ticker,start,end),pred_data(ticker,start,end)) #calling the function we want to predict the target security

Epoch 1/20
32/32 [==============================] - 1s 13ms/step - loss: 19381.1489 - mape: 38.0612 - val_loss: 232.8589 - val_mape: 5.9501
Epoch 2/20
32/32 [==============================] - 0s 5ms/step - loss: 93.3479 - mape: 3.0675 - val_loss: 29.4498 - val_mape: 1.5425
Epoch 3/20
32/32 [==============================] - 0s 5ms/step - loss: 31.6064 - mape: 1.5700 - val_loss: 36.4665 - val_mape: 1.8601
Epoch 4/20
32/32 [==============================] - 0s 5ms/step - loss: 39.5925 - mape: 1.7356 - val_loss: 26.1298 - val_mape: 1.3861
Epoch 5/20
32/32 [==============================] - 0s 5ms/step - loss: 25.1702 - mape: 1.3176 - val_loss: 25.9354 - val_mape: 1.3165
Epoch 6/20
32/32 [==============================] - 0s 12ms/step - loss: 24.9681 - mape: 1.2241 - val_loss: 23.2974 - val_mape: 1.2053
Epoch 7/20
32/32 [==============================] - 0s 6ms/step - loss: 19.0689 - mape: 1.1316 - val_loss: 23.7548 - val_mape: 1.3021
Epoch 8/20
32/32 [==============================] - 0s 